#  <font color='red'>Capstone Project Week3 Part1</font>

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

#### <font color='red'>Scrape postal codes of Canada from Wikipedia and make a DataFrame.</font>

In [2]:
req = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(req.content,'lxml')
table = soup.find_all('table')[0]
df = pd.read_html(str(table))
neighborhood=pd.DataFrame(df[0])
neighborhood

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


#### <font color='red'>Drop cells with a borough that is 'Not assigned'.</font>

In [3]:
neighborhood = neighborhood[neighborhood.Borough != 'Not assigned']
neighborhood

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


#### <font color='red'>If a cell has a borough but a 'Not assigned' neighborhood, then the neighborhood will be the same as the borough.</font>

In [4]:
neighborhood['Neighbourhood'].replace('Not assigned', neighborhood['Borough'])
neighborhood.reset_index(drop=True, inplace=True)
neighborhood

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [5]:
neighborhood.shape

(103, 3)

### <font color='red'>Original data has 180 rows. After dropping "Not assigned" Borough, data has 103 rows. There's no rows that have a Borough but a Not assigned neighborhood.</font>


# <font color='red'>Capstone Project Week3 Part2</font>

In [6]:
!pip install geocoder
import geocoder

#### <font color='red'>Get the list of latitude and longitude coordinates of a given postal code.</font>

In [7]:
lat_lng_coords = None
lati = []
longti = []
i = 0

for postal_code in neighborhood['Postal Code']:
  while(lat_lng_coords is None):
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(neighborhood.loc[i, 'Postal Code']))
    lat_lng_coords = g.latlng

  #print('Postal code = {0}, Latitude = {1}, Longtitude = {2}'.format(postal_code, lat_lng_coords[0], lat_lng_coords[1]))
  lati.append(lat_lng_coords[0])
  longti.append(lat_lng_coords[1])
  i = i + 1
  lat_lng_coords = None

#### <font color='red'>Add latitude and longitude to DataFrame.</font>

In [8]:
neighborhood['Latitude'] = lati
neighborhood['Longtitude'] = longti
neighborhood.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


,Postal Code,Borough,Neighbourhood,Latitude,Longtitude
0,M3A,North York,Parkwoods,43.75245,-79.32991
1,M4A,North York,Victoria Village,43.73057,-79.31306
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72327,-79.45042
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66253,-79.39188


In [9]:
neighborhood.shape

(103, 5)

# <font color='red'>Capstone Project Week3 Part3</font>

#### <font color='red'>Work with only boroughs that contain the word Toronto.</font>

In [10]:
tronto_data = neighborhood[neighborhood['Borough'].str.contains('Toronto')].reset_index(drop=True)
tronto_data.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longtitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66253,-79.39188
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.65739,-79.37804
3,M5C,Downtown Toronto,St. James Town,43.65215,-79.37587
4,M4E,East Toronto,The Beaches,43.67709,-79.29547


#### <font color='red'>Get geograpical coordinate of Toronto.</font>

In [11]:
from geopy.geocoders import Nominatim

address = 'Toronto, Ontario'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


#### <font color='red'>Get a map of Tronto and add markers to the map.</font>

In [12]:
!pip install folium
import folium

map_tronto = folium.Map(location=[latitude, longitude], zoom_start=12)

for lat, lng, borough, neighbor in zip(tronto_data['Latitude'], tronto_data['Longtitude'], tronto_data['Borough'], tronto_data['Neighbourhood']):
    label = '{}, {}'.format(neighbor, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_tronto)  
    
map_tronto

#### <font color='red'>Create a function to explore the neighborhoods in Tronto.</font>

In [13]:
# The code was removed by Watson Studio for sharing.

In [14]:
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### <font color='red'>Get venues DataFrame.</font>

In [16]:
tronto_venues = getNearbyVenues(names=tronto_data['Neighbourhood'],
                                   latitudes=tronto_data['Latitude'],
                                   longitudes=tronto_data['Longtitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

In [17]:
print(tronto_venues.shape)
tronto_venues.head()

(1738, 7)


,Neighbourhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65512,-79.36264,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65512,-79.36264,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65512,-79.36264,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot
3,"Regent Park, Harbourfront",43.65512,-79.36264,The Yoga Lounge,43.655515,-79.364955,Yoga Studio
4,"Regent Park, Harbourfront",43.65512,-79.36264,Body Blitz Spa East,43.654735,-79.359874,Spa


In [18]:
tronto_venues.groupby('Neighbourhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Berczy Park,60,60,60,60,60,60
"Brockton, Parkdale Village, Exhibition Place",85,85,85,85,85,85
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",100,100,100,100,100,100
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",76,76,76,76,76,76
Central Bay Street,76,76,76,76,76,76
Christie,11,11,11,11,11,11
Church and Wellesley,79,79,79,79,79,79
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,26,26,26,26,26,26


In [19]:
print('There are {} uniques categories.'.format(len(tronto_venues['Venue Category'].unique())))

There are 228 uniques categories.


#### <font color='red'>Create a new DataFrame with one-hot-encoded 'Venue Category'.</font>

In [20]:
# one hot encoding
tronto_onehot = pd.get_dummies(tronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
tronto_onehot['Neighbourhood'] = tronto_venues['Neighbourhood']

# move neighborhood column to the first column
fixed_columns = [tronto_onehot.columns[-1]] + list(tronto_onehot.columns[:-1])
tronto_onehot = tronto_onehot[fixed_columns]
tronto_onehot.head()

,Neighbourhood,Accessories Store,Afghan Restaurant,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
tronto_onehot.shape

(1738, 229)

#### <font color='red'>Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category.</font>

In [22]:
tronto_grouped = tronto_onehot.groupby('Neighbourhood').mean().reset_index()
tronto_grouped

,Neighbourhood,Accessories Store,Afghan Restaurant,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.016667,0.000000,0.016667,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.016667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.016667
1,"Brockton, Parkdale Village, Exhibition Place",0.011765,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.023529,0.011765,...,0.000000,0.000000,0.011765,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011765
2,"Business reply mail Processing Centre, South C...",0.000000,0.000000,0.020000,0.000000,0.000000,0.010000,0.000000,0.010000,0.030000,...,0.000000,0.000000,0.020000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013158,...,0.000000,0.013158,0.000000,0.013158,0.000000,0.000000,0.000000,0.000000,0.000000,0.013158
4,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.013158,0.013158,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.013158,0.013158,0.013158,0.000000,0.000000,0.000000
5,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Church and Wellesley,0.000000,0.012658,0.012658,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.012658,0.000000,0.000000,0.000000,0.012658
7,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.030000,0.000000,0.000000,0.010000,0.000000,0.000000,0.010000,...,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.010000
8,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [23]:
tronto_grouped.shape

(38, 229)

#### <font color='red'>Print each neighborhood along with the top 5 most common venues.</font>

In [24]:
num_top_venues = 5

for hood in tronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = tronto_grouped[tronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
            venue  freq
0     Coffee Shop  0.08
1    Cocktail Bar  0.03
2  Breakfast Spot  0.03
3     Cheese Shop  0.03
4        Beer Bar  0.03


----Brockton, Parkdale Village, Exhibition Place----
            venue  freq
0     Coffee Shop  0.06
1            Café  0.06
2             Bar  0.06
3      Restaurant  0.05
4  Sandwich Place  0.04


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
              venue  freq
0       Coffee Shop  0.09
1             Hotel  0.05
2        Restaurant  0.04
3              Café  0.04
4  Asian Restaurant  0.03


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                venue  freq
0  Italian Restaurant  0.08
1                Café  0.07
2         Coffee Shop  0.07
3   French Restaurant  0.04
4                 Bar  0.04


----Central Bay Street----
            venue  freq
0     Coffee Shop  0.12
1  Clothing Store  0.05


#### <font color='red'>Define a function to sort the venues in descending order.</font>

In [25]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### <font color='red'>Create the new dataframe and display the top 10 venues for each neighborhood</font>

In [26]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = tronto_grouped['Neighbourhood']

for ind in np.arange(tronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Bakery,Beer Bar,Breakfast Spot,Restaurant,Cocktail Bar,Seafood Restaurant,Cheese Shop,Farmers Market,Yoga Studio
1,"Brockton, Parkdale Village, Exhibition Place",Coffee Shop,Bar,Café,Restaurant,Gift Shop,Sandwich Place,Nightclub,Supermarket,Japanese Restaurant,Furniture / Home Store
2,"Business reply mail Processing Centre, South C...",Coffee Shop,Hotel,Restaurant,Café,Italian Restaurant,Bar,Asian Restaurant,Seafood Restaurant,Pub,Steakhouse
3,"CN Tower, King and Spadina, Railway Lands, Har...",Italian Restaurant,Café,Coffee Shop,French Restaurant,Park,Bar,Sandwich Place,Restaurant,Speakeasy,Bakery
4,Central Bay Street,Coffee Shop,Clothing Store,Sandwich Place,Bubble Tea Shop,Café,Plaza,Restaurant,Sushi Restaurant,Cosmetics Shop,Bookstore


#### <font color='red'>Run k-means clustering with top 10 venues DataFrame.</font>

In [27]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

tronto_grouped_clustering = tronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:20] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 1, 0],
      dtype=int32)

#### <font color='red'>Merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood.</font>

In [28]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

tronto_merged = tronto_data
tronto_merged = tronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')
tronto_merged = tronto_merged.dropna(how='any')
tronto_merged = tronto_merged.astype({'Cluster Labels': 'int32'})

tronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighbourhood,Latitude,Longtitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264,0,Coffee Shop,Breakfast Spot,Theater,Spa,Food Truck,Event Space,Electronics Store,Restaurant,Bakery,Italian Restaurant
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66253,-79.39188,0,Coffee Shop,Sandwich Place,Gastropub,Theater,Burrito Place,Café,Falafel Restaurant,Fried Chicken Joint,Bank,Italian Restaurant
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.65739,-79.37804,0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Japanese Restaurant,Diner,Bubble Tea Shop,Bookstore,Ramen Restaurant,Fast Food Restaurant
3,M5C,Downtown Toronto,St. James Town,43.65215,-79.37587,0,Coffee Shop,Cocktail Bar,Clothing Store,Hotel,Gastropub,Restaurant,Café,Cosmetics Shop,Beer Bar,Lingerie Store
4,M4E,East Toronto,The Beaches,43.67709,-79.29547,3,Health Food Store,Pub,Trail,Neighborhood,Eastern European Restaurant,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant


#### <font color='red'>Create clusters map.</font>

In [29]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tronto_merged['Latitude'], tronto_merged['Longtitude'], tronto_merged['Neighbourhood'], tronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### <font color='red'>Cluster 1</font>

In [30]:
tronto_merged.loc[tronto_merged['Cluster Labels'] == 0, tronto_merged.columns[[1] + list(range(5, tronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0,Coffee Shop,Breakfast Spot,Theater,Spa,Food Truck,Event Space,Electronics Store,Restaurant,Bakery,Italian Restaurant
1,Downtown Toronto,0,Coffee Shop,Sandwich Place,Gastropub,Theater,Burrito Place,Café,Falafel Restaurant,Fried Chicken Joint,Bank,Italian Restaurant
2,Downtown Toronto,0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Japanese Restaurant,Diner,Bubble Tea Shop,Bookstore,Ramen Restaurant,Fast Food Restaurant
3,Downtown Toronto,0,Coffee Shop,Cocktail Bar,Clothing Store,Hotel,Gastropub,Restaurant,Café,Cosmetics Shop,Beer Bar,Lingerie Store
5,Downtown Toronto,0,Coffee Shop,Bakery,Beer Bar,Breakfast Spot,Restaurant,Cocktail Bar,Seafood Restaurant,Cheese Shop,Farmers Market,Yoga Studio
6,Downtown Toronto,0,Coffee Shop,Clothing Store,Sandwich Place,Bubble Tea Shop,Café,Plaza,Restaurant,Sushi Restaurant,Cosmetics Shop,Bookstore
7,Downtown Toronto,0,Café,Grocery Store,Playground,Baby Store,Candy Store,Italian Restaurant,Coffee Shop,Athletics & Sports,Escape Room,Ethiopian Restaurant
8,Downtown Toronto,0,Coffee Shop,Hotel,Café,Restaurant,Gym,American Restaurant,Salad Place,Japanese Restaurant,Steakhouse,Asian Restaurant
9,West Toronto,0,Park,Grocery Store,Bakery,Pharmacy,Middle Eastern Restaurant,Smoke Shop,Café,Brazilian Restaurant,Furniture / Home Store,Bar
10,Downtown Toronto,0,Coffee Shop,Hotel,Restaurant,Plaza,Japanese Restaurant,Aquarium,Deli / Bodega,Park,Boat or Ferry,Shopping Mall


#### <font color='red'>Cluster 2</font>

In [31]:
tronto_merged.loc[tronto_merged['Cluster Labels'] == 1, tronto_merged.columns[[1] + list(range(5, tronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Central Toronto,1,Bus Line,Swim School,Yoga Studio,Electronics Store,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant


#### <font color='red'>Cluster 3</font>

In [32]:
tronto_merged.loc[tronto_merged['Cluster Labels'] == 2, tronto_merged.columns[[1] + list(range(5, tronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,Central Toronto,2,French Restaurant,Park,Yoga Studio,Eastern European Restaurant,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant


#### <font color='red'>Cluster 4</font>

In [33]:
tronto_merged.loc[tronto_merged['Cluster Labels'] == 3, tronto_merged.columns[[1] + list(range(5, tronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,East Toronto,3,Health Food Store,Pub,Trail,Neighborhood,Eastern European Restaurant,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant


#### <font color='red'>Cluster 5</font>

In [34]:
tronto_merged.loc[tronto_merged['Cluster Labels'] == 4, tronto_merged.columns[[1] + list(range(5, tronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,Central Toronto,4,Playground,Gym Pool,Park,Eastern European Restaurant,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
33,Downtown Toronto,4,Tennis Court,Playground,Park,Shop & Service,Bike Trail,Eastern European Restaurant,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm


### <font color='red'>If I were to go to trip to Tronto, my best cluster will be 5!</font>
